In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.models.resnet import ResNet50_Weights

from tqdm import tqdm
from datetime import datetime
import pytz

import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

# 1. Train Data 확인
---
## 1-1. 데이터 Meta data 확인

In [2]:
TRAIN_FILE_NAME = "train.csv"
DATA_PATH = "../data"

file_path = os.path.join(DATA_PATH, TRAIN_FILE_NAME)
if os.path.exists(file_path):
    train_df = pd.read_csv(file_path)
else:
    print(f"There is no file: {file_path}")

In [4]:
train_df.columns

Index(['Molecule ChEMBL ID', 'Standard Type', 'Standard Relation',
       'Standard Value', 'Standard Units', 'pChEMBL Value', 'Assay ChEMBL ID',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'IC50_nM', 'pIC50', 'Smiles'],
      dtype='object')

In [3]:
import sys
import os

# 현재 노트북의 디렉터리를 기준으로 루트 디렉터리 경로를 추가
project_root = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')), ''))
print(project_root)
sys.path.append(project_root)

# 이제 모듈을 임포트할 수 있습니다
from dataset import SimpleDNNPreprocess, SimpleDNNDataset

/Users/jsh/Projects/dacon/ic50-prediction


In [4]:
dnn_preprocess = SimpleDNNPreprocess(DATA_PATH)
df, test = dnn_preprocess.train_df, dnn_preprocess.test_df

2024-09-04 14:19:18.975 | INFO     | dataset:_load_datas:28 - [Preprocess] start loading datas...
2024-09-04 14:19:18.985 | INFO     | dataset:_load_datas:31 - [Preprocess] end loading datas...
2024-09-04 14:19:18.986 | INFO     | dataset:_preprocess:82 - [SimpleDNNPreprocess] start preprocess train data...
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use MorganGenerator
[14:19:28] DEPRECATION WARNING: please use

In [24]:
df.head()
df['img'] = df['img'].apply(lambda i: np.array(i))

In [25]:
df.head()

,IC50_nM,pIC50,Smiles,assay,document,molecule,img,fingerprint
0,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...,4361896,4359855,4443947,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,4345131,4342485,4556091,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,4345131,4342485,4566431,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,4345131,4342485,4545898,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...,4361896,4359855,4448950,"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [32]:
df['combined_array'] = df.apply(lambda row: [row['document'], row['molecule'], row['assay']], axis=1)

In [34]:
df['flatten'] = df['img'].apply(lambda i: i.reshape(-1,))

In [60]:
df['document'] = df['document'].astype(int)
df['molecule'] = df['molecule'].astype(int)
df['assay'] = df['assay'].astype(int)

# `combined_array` 열 생성
df['combined_array'] = df.apply(
    lambda row: [row['document'], row['molecule'], row['assay']] + 
                row['img'].flatten().tolist()
    , axis=1
)
df['combined_array'] = df.apply(lambda row: np.concat([row['combined_array'], row['fingerprint']]), axis=1)

In [64]:
df['combined_array'][0].shape

(272051,)

In [56]:
df['combined_array'] = df.apply(lambda row: np.concat([row['combined_array'], row['fingerprint']]), axis=1)

(272051,)

In [52]:
len(df['combined_array'][0])

272051

In [ ]:
train_df['Molecule ChEMBL ID'].min(), train_df['Molecule ChEMBL ID'].max()

- 2010년 ~ 2023년 5월 까지 데이터

In [ ]:
train_df.describe()

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
sum(train_df['Standard Value'] != train_df['IC50_nM'])

In [ ]:
sum(train_df['pChEMBL Value'] != train_df['pIC50'])

In [ ]:
# 전처리
train_df = train_df.drop(['Standard Value', 'pChEMBL Value'], axis=1)

In [ ]:
train_df.info()

In [ ]:
import math

In [ ]:
def to_pIC(ic50: float) -> float:
    ic50 = 9 - math.log10(ic50)
    return round(round(ic50, ndigits=3), ndigits=2)

In [ ]:
sum(train_df['IC50_nM'].apply(lambda v: to_pIC(v)) != train_df['pIC50'])

- 학습 데이터에 존재하는 pIC 값 변환 로직 확인

In [ ]:
train_df.head()

In [ ]:
# 전처리
train_df = train_df.drop(train_df.columns[train_df.nunique() == 1], axis=1)

- 학습 데이터 칼럼 중 unique 값이 1개인 대상들 drop

In [ ]:
sample = train_df.iloc[1:4]['Smiles']

In [ ]:
sample.iloc[0]

In [ ]:
sample.iloc[1]

In [ ]:
sample.iloc[2]

In [ ]:
sample.apply(lambda v: len(v))

In [ ]:
train_df.info()

In [ ]:
train_df['Molecule ChEMBL ID'].apply(lambda v: v[:6]).unique()

In [ ]:
train_df['Assay ChEMBL ID'].apply(lambda v: v[:6]).unique()

In [ ]:
train_df['Document ChEMBL ID'].apply(lambda v: v[:6]).unique()

In [ ]:
# 전처리
train_df['assay'] = train_df['Assay ChEMBL ID'].apply(lambda v: v[6:])
train_df['document'] = train_df['Document ChEMBL ID'].apply(lambda v: v[6:])
train_df['molecule'] = train_df['Molecule ChEMBL ID'].apply(lambda v: v[6:])

In [ ]:
train_df.head()

- feature 중 Document, Essay 값이 같은 데이터 들의 Smiles 비교

In [ ]:
# SMILES 문자열
smiles_string = 'CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c(N3CCN(C4COC4)CC3)cc2C1=O'

# 분자 객체 생성
molecule = Chem.MolFromSmiles(smiles_string)

# 2D 구조 그리기
image = Draw.MolToImage(molecule)


In [ ]:
def img_of(smiles: str):
    return Draw.MolToImage(Chem.MolFromSmiles(smiles))

In [ ]:
print(img_of(smiles_string))

In [ ]:
# 전처리
train_df['img'] = train_df['Smiles'].apply(lambda x: img_of(x))

In [ ]:
train_df.iloc[1:4]['img'][1]

In [ ]:
train_df.iloc[1:4]['img'][2]

In [ ]:
train_df.iloc[1:4]['img'][3]

아이디어
- smiles를 image로 변환해 해당 이미지의 임베딩을 입력으로 사용 가능

In [ ]:
train_df[['assay', 'document', 'molecule']].nunique()

- CHEMBL 이라는 데이터베이스를 기반으로 한 데이터셋 구성으로 보임
- document 에 여러종류의 assay 가 존재하는 듯?
  - 2024.09.03 14:43 까지 확인한 바론, document는 논문 혹은 발표 단위. assay는 실험 단위. molucule은 화합물 단위.
  - 하나의 document에 여러개의 assay가 포함될 수 있고, 각 assay는 보통 하나의 molecule을 실험하는 듯
- 위 가정이 맞다면 아래와 같은 추론이 가능해보임
  - document 별로 group by 한 document 들의 IC50 평가 값들은 유사한 범위 내에 포함될 확률이 높다 -> 확실하지 않다. document 별 통계값을 확인해볼 수 있을 것 같음.

- assay 와 document가 전체 값에 비해 종류가 적음
- 각 그룹별로 유사한 패턴을 갖지 않는지 확인
- 분자구조들의 유사도를 측정할 수 있으면 좋을 것 같음
- 각 기준으로 그룹화 한 뒤 그룹내의 각 pair들간의 유사도 평균을 산출해 해당 값이 높으면 해당 기준이 분자구조간의 유사도와 연관이 있다는 의미 이므로, 분류 기준값으로 사용해도 될 것 같음
  - https://medium.com/standigm/rdkit%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-chembl-%EB%B6%84%EC%9E%90%EB%93%A4-%EC%82%AC%EC%9D%B4%EC%9D%98-%EC%9C%A0%EC%82%AC%EB%8F%84-%EA%B2%80%EC%83%89-60307ccdb441
- 학습 데이터는 아래 링크 데이터의 메타 정보를 따름: https://www.ebi.ac.uk/chembl/web_components/explore/activities/STATE_ID:RIRpPJ6zOpLv-SJ8HzgSFw%3D%3D

In [ ]:
train_df[['document', 'IC50_nM']].groupby(['document']).agg(['sum', 'mean', 'min', 'max', 'std'])

In [ ]:
train_df[['assay', 'document', 'IC50_nM']].groupby(['document', 'assay']).agg(lambda x: np.mean(x))

In [ ]:
train_df[['assay', 'Smiles']].groupby('assay').agg(lambda x: list(x)).iloc[7, 0]

In [ ]:
train_df[['document', 'Smiles']].groupby('document').agg(lambda x: list(x)).iloc[1, 0]

## 1-2. chEMBL 데이터베이스 확인
- train set 에 포함된 document id, assay id 값을 학습에 사용하기 위해선 TEST 데이터에 대해서도 동일한 데이터를 얻을 수 있어야 함.
  - train 데이터의 Smiles 로 chEMBL 데이터에서 document 와 assay 값을 가져온 뒤 비교 검증
  - 동일한 데이터가 존재한다면 test 데이터에 대해 동일로직 검증
  - 결측치에 대한 처리 로직 고민 필요

### chEMBL schema
- full schema: https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_34_schema.png
- 주요 테이블
  - DOCS: document 정보
  - ASSAYS: assay 정보
  - ASSAY_PARAMETERS: assay 별 측정값 정보 (IC50 값)
  - TISSUE_DICTIONARY: assay에 매핑되는 화합물의 CHEMBL ID 정보

- 데이터베이스는 파일기반 처리가 가능한 sqlite로 처리

In [ ]:
!wget https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_34_sqlite.tar.gz

In [ ]:
!tar -zxvf chembl_34_mysql.tar.gz

---
# 2. 학습 파이프라인 생성

In [ ]:
class DataPreprocess:
    def __init__(self, 
                 data_dir: str, 
                 train_name: str = 'train.csv',
                 test_name: str = 'test.csv'):
        self.data_dir = data_dir
        self.train_name = train_name
        self.test_name = test_name
        self._load_datas()
        self._preprocess()
    
    def _load_datas(self):
        print("[Preprocess] start loading datas...")
        self.train_df = pd.read_csv(os.path.join(self.data_dir, self.train_name))
        self.test_df = pd.read_csv(os.path.join(self.data_dir, self.test_name))
        print("[Preprocess] end loading datas...")

    def _preprocess(self):
        def img_of(smiles: str):
            return Draw.MolToImage(Chem.MolFromSmiles(smiles))
        
        print("[Preprocess] end loading datas...")
        self.train_df['assay'] = self.train_df['Assay ChEMBL ID'].apply(lambda v: v[6:])
        self.train_df['document'] = self.train_df['Document ChEMBL ID'].apply(lambda v: v[6:])
        self.train_df['molecule'] = self.train_df['Molecule ChEMBL ID'].apply(lambda v: v[6:])
        self.train_df = self.train_df.drop(self.train_df.columns[self.train_df.nunique() == 1], axis=1)
        self.train_df['img'] = self.train_df['Smiles'].apply(lambda x: img_of(x))
        self.train_df = self.train_df.drop(['Standard Value', 'pChEMBL Value', 'Assay ChEMBL ID', 'Document ChEMBL ID', 'Molecule ChEMBL ID'], axis=1)
        
        print("[Preprocess] start test datas...")
        self.test_df['img'] = self.test_df['Smiles'].apply(lambda x: img_of(x))
        print("[Preprocess] end loading datas...")

class IC50Dataset(Dataset):
    def __init__(self, data: pd.DataFrame, transform, train: bool = True):
        self.data = data
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        return {
            'X': self.transform(item['img']).view(-1,),
            # 'X': np.array(item['img'], dtype=np.float32).reshape(-1,),
            'Y': item['IC50_nM']
        } if self.train else {
            'X': self.transform(item['img']).view(-1,),
        }

preprocessor = DataPreprocess('../data')

train_df, test_df = preprocessor.train_df, preprocessor.test_df

# 이미지 변환 정의 (300x300 크기로 조정)
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = IC50Dataset(train_df, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32)

test_dataset = IC50Dataset(test_df, transform=transform, train=False)
test_dataloader = DataLoader(test_dataset, batch_size=32)

# ResNet50 사전 훈련된 모델 로드 및 수정
resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
# embedding_size = resnet.fc.in_features  # 수정된 부분
embedding_size = 300 * 300 * 3  # 수정된 부분
resnet.fc = nn.Identity()  # 마지막 fc 레이어 제거하여 임베딩을 출력하도록 설정

# 추가 모델 정의 (임베딩을 입력으로 사용하여 회귀 작업 수행)
class SimpleImageRegressor(nn.Module):
    def __init__(self, embedding_size):
        super(SimpleImageRegressor, self).__init__()
        self.fc = nn.Linear(embedding_size, 1)  # 간단한 회귀 모델

    def forward(self, x):
        return self.fc(x)

# 모델 초기화
model = SimpleImageRegressor(embedding_size)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련 루프
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.
    for data in tqdm(train_dataloader):
        optimizer.zero_grad()
        images, targets = data['X'], data['Y']
        outputs = model(images)
        loss = criterion(outputs.squeeze(), targets.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")

# 훈련이 완료되면 임베딩 추출 및 추가적인 예측 작업에 사용할 수 있습니다.
submission = []
for data in tqdm(test_dataloader):
    model.eval()
    images = data['X']
    outputs = model(images)
    submission.extend(outputs.detach().numpy())

sample_df = pd.read_csv('../data/sample_submission.csv')
sample_df['IC50_nM'] = np.array(submission).reshape(-1)

output_name = datetime.now(pytz.timezone("Asia/Seoul"))
sample_df.to_csv(f'../data/submissions/{output_name}.csv', index=False)

In [ ]:
submission = []
for data in tqdm(test_dataloader):
    model.eval()
    images = data['X']
    outputs = model(images)
    submission.extend(outputs.detach().numpy())

sample_df['IC50_nM'] = np.array(submission).reshape(-1)

In [ ]:
sample_df['IC50_nM'] = np.array(submission).reshape(-1)

In [ ]:
sample_df.head()